# State machine like implementation for grasping

In [1]:
from rlbench.environment import Environment
from rlbench.action_modes import ArmActionMode, ActionMode
from rlbench.observation_config import ObservationConfig
from rlbench.tasks import *
import numpy as np
from pyrep.const import ConfigurationPathAlgorithms as Algos

In [2]:
class StateMachine(object):
    def __init__(self):
        self.env=None
        self.home=None
        self.task=None
        
    def initialize(self):
        DATASET = ''
        obs_config = ObservationConfig()
        obs_config.set_all(True)
        obs_config.left_shoulder_camera.rgb = True
        obs_config.right_shoulder_camera.rgb = True
        action_mode = ActionMode(ArmActionMode.DELTA_EE_POSE)
        self.env = Environment(
            action_mode, DATASET, obs_config, False)
        self.env.launch()
        self.task = self.env.get_task(EmptyContainer)
        self.task.reset()
        self.home = self.get_objects()['large_container'].get_pose()
        self.home[2]+=0.3
        # demos = task.get_demos(3, live_demos=live_demos)
        
    def get_objects(self):
        objs = self.env._scene._active_task.get_base().get_objects_in_tree(exclude_base=True, first_generation_only=False)
        objs_dict = {}
        for obj in objs:
            name = obj.get_name()
            pose = obj.get_pose()
            objs_dict[name] = obj
        return objs_dict

    # Move above object
    def move_to(self, pose, pad=0.05, ignore_collisions=True):
        target_pose = np.copy(pose)
        obs = self.env._scene.get_observation()
        init_pose=obs.gripper_pose
        obs = self.env._scene.get_observation()
        init_pose=obs.gripper_pose
        target_pose[2]+=pad
        path=self.env._robot.arm.get_path(np.array(target_pose[0:3]),quaternion=np.array([0,1,0,0]), trials=1000,ignore_collisions=True, algorithm=Algos.RRTConnect)
        # TODO catch errors and deal with situations when path not found
        return path
    
    def grasp(self,obj):
        # TODO get feedback to check if grasp is successfull
        cond=False
        while not cond:
            cond=self.env._robot.gripper.actuate(0,0.1)
            self.env._scene.step()
        cond = self.env._robot.gripper.grasp(obj)
        return cond
    
    def release(self, obj):
        cond=False
        while not cond:
            cond=self.env._robot.gripper.actuate(1,0.1)
            self.env._scene.step()
        self.env._robot.gripper.release()
    
    def execute(self, path):
        done=False
        path.set_to_start()
        while not done:
            done = path.step()
            a = path.visualize()
            self.env._scene.step()
        return done
    
    def reset(self):
        self.task.reset()
        

In [3]:
# env, task, obs = init()
machine = StateMachine()
machine.initialize()

/home/arcot/.local/lib/python3.7/site-packages/pyrep/pyrep.py:209: UserWarning: Could not change simulation timestep. You may need to change it to "custom dt" using simulation settings dialog.
  warnings.warn('Could not change simulation timestep. You may need '


In [4]:
machine.reset()
objects = ['Shape', 'Shape1', 'Shape3']
for shape in objects:
    # move above object
    objs=machine.get_objects()
    pose=objs[shape].get_pose()
    path=machine.move_to(pose,False)
    machine.execute(path)

    # grasp the object
    cond = machine.grasp(objs[shape])

    # move to home position
    path=machine.move_to(machine.home, 0, True)
    machine.execute(path)

    # move above small container
    obj=machine.get_objects()['small_container0']
    pose = obj.get_pose()
    path=machine.move_to(pose,0.1,True)
    machine.execute(path)

    # release the object
    machine.release(obj)

    # go back to home position
    path=machine.move_to(machine.home,0,True)
    machine.execute(path)

In [5]:
machine.get_objects()

{'large_container': <pyrep.objects.shape.Shape at 0x7f217a3425c0>,
 'small_container0': <pyrep.objects.shape.Shape at 0x7f217a342780>,
 'small_container1': <pyrep.objects.shape.Shape at 0x7f217a3427f0>,
 'Shape': <pyrep.objects.shape.Shape at 0x7f217a342128>,
 'Shape1': <pyrep.objects.shape.Shape at 0x7f217a342748>,
 'Shape3': <pyrep.objects.shape.Shape at 0x7f217a342860>,
 'waypoint0': <pyrep.objects.dummy.Dummy at 0x7f217a3427b8>,
 'waypoint1': <pyrep.objects.dummy.Dummy at 0x7f217a342898>,
 'waypoint2': <pyrep.objects.dummy.Dummy at 0x7f217a342940>,
 'waypoint3': <pyrep.objects.dummy.Dummy at 0x7f217a342978>,
 'spawn_boundary': <pyrep.objects.shape.Shape at 0x7f217a3429e8>,
 'success0': <pyrep.objects.proximity_sensor.ProximitySensor at 0x7f217a342828>,
 'success1': <pyrep.objects.proximity_sensor.ProximitySensor at 0x7f217a3428d0>,
 'Shape0': <pyrep.objects.shape.Shape at 0x7f217a342a20>,
 'Shape2': <pyrep.objects.shape.Shape at 0x7f217a342a58>,
 'Shape4': <pyrep.objects.shape.Shap